In [62]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reproducible.
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(1)

In [63]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from plotly.offline import init_notebook_mode, iplot
import chart_studio.plotly as py
import plotly.graph_objs as go

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

In [64]:
train = pd.read_csv('train.csv', parse_dates=['date'])

In [65]:
test = pd.read_csv('test.csv', parse_dates=['date'])

In [66]:
test

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1
...,...,...,...,...
44995,44995,2018-03-27,10,50
44996,44996,2018-03-28,10,50
44997,44997,2018-03-29,10,50
44998,44998,2018-03-30,10,50


In [67]:
train

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [68]:
train.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [69]:
train.head()


,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [70]:
print('Min date from train set: %s' % train['date'].min().date())
print('Max date from train set: %s' % train['date'].max().date())

Min date from train set: 2013-01-01
Max date from train set: 2017-12-31


In [71]:
lag_size = (test['date'].max().date() - train['date'].max().date()).days
print('Forecast Lag Size', lag_size)

Forecast Lag Size 90


In [11]:
lag_size = 30

## EDA

In [73]:
daily_sales = train.groupby('date',as_index = False)['sales'].sum()

In [74]:
daily_sales

,date,sales
0,2017-01-01,23709
1,2017-01-02,15772
2,2017-01-03,18650
3,2017-01-04,18510
4,2017-01-05,19895
...,...,...
360,2017-12-27,20378
361,2017-12-28,21885
362,2017-12-29,23535
363,2017-12-30,24988


In [75]:
store_daily_sales = train.groupby(['store','date'],as_index=False)['sales'].sum()
store_daily_sales

,store,date,sales
0,1,2017-01-01,2155
1,1,2017-01-02,1441
2,1,2017-01-03,1635
3,1,2017-01-04,1713
4,1,2017-01-05,1858
...,...,...,...
3645,10,2017-12-27,2221
3646,10,2017-12-28,2429
3647,10,2017-12-29,2687
3648,10,2017-12-30,2742


In [76]:
item_daily_sales = train.groupby(['item','date'], as_index = False)['sales'].sum()
item_daily_sales

,item,date,sales
0,1,2017-01-01,205
1,1,2017-01-02,139
2,1,2017-01-03,141
3,1,2017-01-04,155
4,1,2017-01-05,175
...,...,...,...
18245,50,2017-12-27,511
18246,50,2017-12-28,587
18247,50,2017-12-29,596
18248,50,2017-12-30,612


In [77]:
store_item_daily_sales = train.groupby(['date','store','item'],as_index=False)['sales'].sum()
store_item_daily_sales ## No point as we are getting results same as starting dataset

,date,store,item,sales
0,2017-01-01,1,1,19
1,2017-01-01,1,2,40
2,2017-01-01,1,3,36
3,2017-01-01,1,4,21
4,2017-01-01,1,5,25
...,...,...,...,...
182495,2017-12-31,10,46,70
182496,2017-12-31,10,47,22
182497,2017-12-31,10,48,60
182498,2017-12-31,10,49,42


In [78]:
daily_sales_sc = go.Scatter(x=daily_sales['date'], y=daily_sales['sales'])
layout = go.Layout(title= 'Daily Sales', xaxis = dict(title='Date'), yaxis = dict(title='Sales'))
fig = go.Figure(data = [daily_sales_sc], layout=layout )
iplot(fig)

In [79]:
store_daily_sales_sc = []
for store in store_daily_sales['store'].unique():
    current = store_daily_sales[(store_daily_sales['store'] == store)]
    store_daily_sales_sc.append(go.Scatter(x=current['date'] , y=current['sales'], name =('Store %s' % store)))
    
    layout= go.Layout(title= 'Store Daily Sales' , xaxis = dict(title='Date'), yaxis=dict(title='Sales'))
    fig = go.Figure(data=store_daily_sales_sc, layout=layout)
    iplot(fig)

In [80]:
store_daily_sales_sc = []
for store in store_daily_sales['store'].unique():
    current = store_daily_sales[(store_daily_sales['store'] == store)]
    store_daily_sales_sc.append(go.Scatter(x=current['date'] , y=current['sales'], name =('Store %s' % store)))
    
layout= go.Layout(title= 'Store Daily Sales' , xaxis = dict(title='Date'), yaxis=dict(title='Sales'))
fig = go.Figure(data=store_daily_sales_sc, layout=layout)
iplot(fig)

In [81]:
item_daily_sales_sc = []
for item in item_daily_sales['item'].unique():
    current_item_daily_sales = item_daily_sales[(item_daily_sales['item'] == item)]
    item_daily_sales_sc.append(go.Scatter(x=current_item_daily_sales['date'], y=current_item_daily_sales['sales'], name=('Item %s' % item)))

layout = go.Layout(title='Item daily sales', xaxis=dict(title='Date'), yaxis=dict(title='Sales'))
fig = go.Figure(data=item_daily_sales_sc, layout=layout)
iplot(fig)

Sub-sample train set to get only the last year of data and reduce training time

In [72]:
train

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [73]:
train

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


### Rearrange dataset so we can apply shift methods

In [74]:
train_gp = train.sort_values('date').groupby(['item', 'store', 'date'], as_index=False)
train_gp = train_gp.agg({'sales':['mean']})
train_gp.columns = ['item', 'store', 'date', 'sales']
train_gp.head()

,item,store,date,sales
0,1,1,2013-01-01,13
1,1,1,2013-01-02,11
2,1,1,2013-01-03,14
3,1,1,2013-01-04,13
4,1,1,2013-01-05,10


In [75]:
train_gp[29:65]

,item,store,date,sales
29,1,1,2013-01-30,9
30,1,1,2013-01-31,13
31,1,1,2013-02-01,11
32,1,1,2013-02-02,21
33,1,1,2013-02-03,15
34,1,1,2013-02-04,14
35,1,1,2013-02-05,9
36,1,1,2013-02-06,10
37,1,1,2013-02-07,13
38,1,1,2013-02-08,11


### Transform the data into a time series problem

In [76]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

### We will use the current timestep and the last 29 to forecast 90 days ahead

In [77]:
window = 29
lag = lag_size
series = series_to_supervised(train_gp.drop('date', axis=1), window=window, lag=lag)
series.head()

,item(t-29),store(t-29),sales(t-29),item(t-28),store(t-28),sales(t-28),item(t-27),store(t-27),sales(t-27),item(t-26),...,sales(t-2),item(t-1),store(t-1),sales(t-1),item(t),store(t),sales(t),item(t+90),store(t+90),sales(t+90)
29,1.0,1.0,13.0,1.0,1.0,11.0,1.0,1.0,14.0,1.0,...,11.0,1.0,1.0,6.0,1,1,9,1.0,1.0,15.0
30,1.0,1.0,11.0,1.0,1.0,14.0,1.0,1.0,13.0,1.0,...,6.0,1.0,1.0,9.0,1,1,13,1.0,1.0,16.0
31,1.0,1.0,14.0,1.0,1.0,13.0,1.0,1.0,10.0,1.0,...,9.0,1.0,1.0,13.0,1,1,11,1.0,1.0,23.0
32,1.0,1.0,13.0,1.0,1.0,10.0,1.0,1.0,12.0,1.0,...,13.0,1.0,1.0,11.0,1,1,21,1.0,1.0,22.0
33,1.0,1.0,10.0,1.0,1.0,12.0,1.0,1.0,10.0,1.0,...,11.0,1.0,1.0,21.0,1,1,15,1.0,1.0,20.0


In [78]:
series

,item(t-29),store(t-29),sales(t-29),item(t-28),store(t-28),sales(t-28),item(t-27),store(t-27),sales(t-27),item(t-26),...,sales(t-2),item(t-1),store(t-1),sales(t-1),item(t),store(t),sales(t),item(t+90),store(t+90),sales(t+90)
29,1.0,1.0,13.0,1.0,1.0,11.0,1.0,1.0,14.0,1.0,...,11.0,1.0,1.0,6.0,1,1,9,1.0,1.0,15.0
30,1.0,1.0,11.0,1.0,1.0,14.0,1.0,1.0,13.0,1.0,...,6.0,1.0,1.0,9.0,1,1,13,1.0,1.0,16.0
31,1.0,1.0,14.0,1.0,1.0,13.0,1.0,1.0,10.0,1.0,...,9.0,1.0,1.0,13.0,1,1,11,1.0,1.0,23.0
32,1.0,1.0,13.0,1.0,1.0,10.0,1.0,1.0,12.0,1.0,...,13.0,1.0,1.0,11.0,1,1,21,1.0,1.0,22.0
33,1.0,1.0,10.0,1.0,1.0,12.0,1.0,1.0,10.0,1.0,...,11.0,1.0,1.0,21.0,1,1,15,1.0,1.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912905,50.0,10.0,79.0,50.0,10.0,89.0,50.0,10.0,96.0,50.0,...,79.0,50.0,10.0,80.0,50,10,82,50.0,10.0,63.0
912906,50.0,10.0,89.0,50.0,10.0,96.0,50.0,10.0,92.0,50.0,...,80.0,50.0,10.0,82.0,50,10,90,50.0,10.0,59.0
912907,50.0,10.0,96.0,50.0,10.0,92.0,50.0,10.0,102.0,50.0,...,82.0,50.0,10.0,90.0,50,10,103,50.0,10.0,74.0
912908,50.0,10.0,92.0,50.0,10.0,102.0,50.0,10.0,60.0,50.0,...,90.0,50.0,10.0,103.0,50,10,99,50.0,10.0,62.0


### Drop rows with different item or store values than the shifted columns


In [79]:
last_item = 'item(t-%d)' % window
last_store = 'store(t-%d)' % window
series = series[(series['store(t)'] == series[last_store])]
series = series[(series['item(t)'] == series[last_item])]

In [80]:
series

,item(t-29),store(t-29),sales(t-29),item(t-28),store(t-28),sales(t-28),item(t-27),store(t-27),sales(t-27),item(t-26),...,sales(t-2),item(t-1),store(t-1),sales(t-1),item(t),store(t),sales(t),item(t+90),store(t+90),sales(t+90)
29,1.0,1.0,13.0,1.0,1.0,11.0,1.0,1.0,14.0,1.0,...,11.0,1.0,1.0,6.0,1,1,9,1.0,1.0,15.0
30,1.0,1.0,11.0,1.0,1.0,14.0,1.0,1.0,13.0,1.0,...,6.0,1.0,1.0,9.0,1,1,13,1.0,1.0,16.0
31,1.0,1.0,14.0,1.0,1.0,13.0,1.0,1.0,10.0,1.0,...,9.0,1.0,1.0,13.0,1,1,11,1.0,1.0,23.0
32,1.0,1.0,13.0,1.0,1.0,10.0,1.0,1.0,12.0,1.0,...,13.0,1.0,1.0,11.0,1,1,21,1.0,1.0,22.0
33,1.0,1.0,10.0,1.0,1.0,12.0,1.0,1.0,10.0,1.0,...,11.0,1.0,1.0,21.0,1,1,15,1.0,1.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912905,50.0,10.0,79.0,50.0,10.0,89.0,50.0,10.0,96.0,50.0,...,79.0,50.0,10.0,80.0,50,10,82,50.0,10.0,63.0
912906,50.0,10.0,89.0,50.0,10.0,96.0,50.0,10.0,92.0,50.0,...,80.0,50.0,10.0,82.0,50,10,90,50.0,10.0,59.0
912907,50.0,10.0,96.0,50.0,10.0,92.0,50.0,10.0,102.0,50.0,...,82.0,50.0,10.0,90.0,50,10,103,50.0,10.0,74.0
912908,50.0,10.0,92.0,50.0,10.0,102.0,50.0,10.0,60.0,50.0,...,90.0,50.0,10.0,103.0,50,10,99,50.0,10.0,62.0


### Remove unwanted columns

In [81]:
columns_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['item', 'store']]
for i in range(window, 0, -1):
    columns_to_drop += [('%s(t-%d)' % (col, i)) for col in ['item', 'store']]
series.drop(columns_to_drop, axis=1, inplace=True)
series.drop(['item(t)', 'store(t)'], axis=1, inplace=True)


In [82]:
series

,sales(t-29),sales(t-28),sales(t-27),sales(t-26),sales(t-25),sales(t-24),sales(t-23),sales(t-22),sales(t-21),sales(t-20),...,sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t),sales(t+90)
29,13.0,11.0,14.0,13.0,10.0,12.0,10.0,9.0,12.0,9.0,...,7.0,9.0,8.0,14.0,12.0,12.0,11.0,6.0,9,15.0
30,11.0,14.0,13.0,10.0,12.0,10.0,9.0,12.0,9.0,9.0,...,9.0,8.0,14.0,12.0,12.0,11.0,6.0,9.0,13,16.0
31,14.0,13.0,10.0,12.0,10.0,9.0,12.0,9.0,9.0,7.0,...,8.0,14.0,12.0,12.0,11.0,6.0,9.0,13.0,11,23.0
32,13.0,10.0,12.0,10.0,9.0,12.0,9.0,9.0,7.0,10.0,...,14.0,12.0,12.0,11.0,6.0,9.0,13.0,11.0,21,22.0
33,10.0,12.0,10.0,9.0,12.0,9.0,9.0,7.0,10.0,12.0,...,12.0,12.0,11.0,6.0,9.0,13.0,11.0,21.0,15,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912905,79.0,89.0,96.0,92.0,102.0,60.0,81.0,79.0,78.0,97.0,...,72.0,89.0,97.0,97.0,99.0,63.0,79.0,80.0,82,63.0
912906,89.0,96.0,92.0,102.0,60.0,81.0,79.0,78.0,97.0,93.0,...,89.0,97.0,97.0,99.0,63.0,79.0,80.0,82.0,90,59.0
912907,96.0,92.0,102.0,60.0,81.0,79.0,78.0,97.0,93.0,96.0,...,97.0,97.0,99.0,63.0,79.0,80.0,82.0,90.0,103,74.0
912908,92.0,102.0,60.0,81.0,79.0,78.0,97.0,93.0,96.0,74.0,...,97.0,99.0,63.0,79.0,80.0,82.0,90.0,103.0,99,62.0


### Train/Validation Split

In [83]:
# Label
labels_col = 'sales(t+%d)' % lag_size
labels = series[labels_col]
series = series.drop(labels_col, axis=1)

X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=0.4, random_state=0)
print('Train set shape', X_train.shape)
print('Validation set shape', X_valid.shape)
X_train.head()

Train set shape (539046, 30)
Validation set shape (359364, 30)


,sales(t-29),sales(t-28),sales(t-27),sales(t-26),sales(t-25),sales(t-24),sales(t-23),sales(t-22),sales(t-21),sales(t-20),...,sales(t-9),sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t)
418620,35.0,32.0,36.0,20.0,22.0,27.0,26.0,38.0,34.0,34.0,...,35.0,26.0,45.0,38.0,29.0,26.0,30.0,27.0,25.0,41
224141,105.0,104.0,121.0,87.0,89.0,121.0,106.0,115.0,126.0,147.0,...,106.0,117.0,93.0,131.0,78.0,105.0,101.0,114.0,127.0,115
38187,39.0,40.0,35.0,45.0,41.0,58.0,36.0,47.0,41.0,43.0,...,46.0,43.0,40.0,41.0,59.0,43.0,68.0,46.0,39.0,37
885034,26.0,35.0,34.0,19.0,14.0,26.0,21.0,32.0,24.0,35.0,...,23.0,42.0,35.0,35.0,19.0,18.0,26.0,31.0,32.0,34
199559,63.0,85.0,85.0,91.0,98.0,100.0,99.0,67.0,86.0,77.0,...,110.0,73.0,85.0,102.0,94.0,85.0,106.0,128.0,67.0,92


In [84]:
Y_train

array([45., 88., 43., ..., 55., 76., 55.])

In [85]:
len(Y_valid)

359364

In [86]:
X_train

,sales(t-29),sales(t-28),sales(t-27),sales(t-26),sales(t-25),sales(t-24),sales(t-23),sales(t-22),sales(t-21),sales(t-20),...,sales(t-9),sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t)
418620,35.0,32.0,36.0,20.0,22.0,27.0,26.0,38.0,34.0,34.0,...,35.0,26.0,45.0,38.0,29.0,26.0,30.0,27.0,25.0,41
224141,105.0,104.0,121.0,87.0,89.0,121.0,106.0,115.0,126.0,147.0,...,106.0,117.0,93.0,131.0,78.0,105.0,101.0,114.0,127.0,115
38187,39.0,40.0,35.0,45.0,41.0,58.0,36.0,47.0,41.0,43.0,...,46.0,43.0,40.0,41.0,59.0,43.0,68.0,46.0,39.0,37
885034,26.0,35.0,34.0,19.0,14.0,26.0,21.0,32.0,24.0,35.0,...,23.0,42.0,35.0,35.0,19.0,18.0,26.0,31.0,32.0,34
199559,63.0,85.0,85.0,91.0,98.0,100.0,99.0,67.0,86.0,77.0,...,110.0,73.0,85.0,102.0,94.0,85.0,106.0,128.0,67.0,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896639,53.0,40.0,66.0,65.0,30.0,53.0,52.0,41.0,54.0,51.0,...,41.0,69.0,68.0,81.0,71.0,66.0,58.0,66.0,67.0,53
154780,58.0,33.0,40.0,54.0,25.0,34.0,42.0,54.0,47.0,53.0,...,39.0,50.0,56.0,46.0,50.0,38.0,28.0,40.0,44.0,51
119866,26.0,29.0,43.0,43.0,34.0,36.0,42.0,32.0,48.0,42.0,...,48.0,43.0,34.0,29.0,55.0,44.0,42.0,68.0,35.0,39
442876,122.0,120.0,131.0,86.0,92.0,82.0,105.0,101.0,104.0,114.0,...,111.0,94.0,103.0,120.0,81.0,81.0,91.0,105.0,98.0,108


In [87]:
X_train.columns

Index(['sales(t-29)', 'sales(t-28)', 'sales(t-27)', 'sales(t-26)',
       'sales(t-25)', 'sales(t-24)', 'sales(t-23)', 'sales(t-22)',
       'sales(t-21)', 'sales(t-20)', 'sales(t-19)', 'sales(t-18)',
       'sales(t-17)', 'sales(t-16)', 'sales(t-15)', 'sales(t-14)',
       'sales(t-13)', 'sales(t-12)', 'sales(t-11)', 'sales(t-10)',
       'sales(t-9)', 'sales(t-8)', 'sales(t-7)', 'sales(t-6)', 'sales(t-5)',
       'sales(t-4)', 'sales(t-3)', 'sales(t-2)', 'sales(t-1)', 'sales(t)'],
      dtype='object')

### MLP for Time Series Forecasting

In [88]:
epochs = 40
batch = 256
lr = 0.0003
adam = optimizers.Adam(lr)

In [89]:
model_mlp = Sequential()
model_mlp.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))
model_mlp.add(Dense(1))
model_mlp.compile(loss='mse', optimizer=adam)
model_mlp.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               3100      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 3,201
Trainable params: 3,201
Non-trainable params: 0
_________________________________________________________________


In [90]:
mlp_history = model_mlp.fit(X_train.values, Y_train, validation_data=(X_valid.values, Y_valid), epochs=epochs, verbose=2)

Epoch 1/40
16846/16846 - 35s - loss: 296.5333 - val_loss: 284.5349
Epoch 2/40
16846/16846 - 35s - loss: 285.2821 - val_loss: 283.1136
Epoch 3/40
16846/16846 - 34s - loss: 282.4951 - val_loss: 284.5041
Epoch 4/40
16846/16846 - 34s - loss: 281.2334 - val_loss: 281.4929
Epoch 5/40
16846/16846 - 34s - loss: 279.9447 - val_loss: 278.4334
Epoch 6/40
16846/16846 - 34s - loss: 278.8258 - val_loss: 280.6985
Epoch 7/40
16846/16846 - 34s - loss: 277.6136 - val_loss: 276.3546
Epoch 8/40
16846/16846 - 34s - loss: 276.6361 - val_loss: 275.5880
Epoch 9/40
16846/16846 - 35s - loss: 275.7032 - val_loss: 274.4774
Epoch 10/40
16846/16846 - 34s - loss: 274.9640 - val_loss: 276.4727
Epoch 11/40
16846/16846 - 34s - loss: 274.2777 - val_loss: 274.9925
Epoch 12/40
16846/16846 - 34s - loss: 273.8915 - val_loss: 272.1194
Epoch 13/40
16846/16846 - 34s - loss: 273.5566 - val_loss: 272.6711
Epoch 14/40
16846/16846 - 34s - loss: 273.1599 - val_loss: 271.9370
Epoch 15/40
16846/16846 - 34s - loss: 272.8426 - val_loss

In [91]:
X_train

,sales(t-29),sales(t-28),sales(t-27),sales(t-26),sales(t-25),sales(t-24),sales(t-23),sales(t-22),sales(t-21),sales(t-20),...,sales(t-9),sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t)
418620,35.0,32.0,36.0,20.0,22.0,27.0,26.0,38.0,34.0,34.0,...,35.0,26.0,45.0,38.0,29.0,26.0,30.0,27.0,25.0,41
224141,105.0,104.0,121.0,87.0,89.0,121.0,106.0,115.0,126.0,147.0,...,106.0,117.0,93.0,131.0,78.0,105.0,101.0,114.0,127.0,115
38187,39.0,40.0,35.0,45.0,41.0,58.0,36.0,47.0,41.0,43.0,...,46.0,43.0,40.0,41.0,59.0,43.0,68.0,46.0,39.0,37
885034,26.0,35.0,34.0,19.0,14.0,26.0,21.0,32.0,24.0,35.0,...,23.0,42.0,35.0,35.0,19.0,18.0,26.0,31.0,32.0,34
199559,63.0,85.0,85.0,91.0,98.0,100.0,99.0,67.0,86.0,77.0,...,110.0,73.0,85.0,102.0,94.0,85.0,106.0,128.0,67.0,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896639,53.0,40.0,66.0,65.0,30.0,53.0,52.0,41.0,54.0,51.0,...,41.0,69.0,68.0,81.0,71.0,66.0,58.0,66.0,67.0,53
154780,58.0,33.0,40.0,54.0,25.0,34.0,42.0,54.0,47.0,53.0,...,39.0,50.0,56.0,46.0,50.0,38.0,28.0,40.0,44.0,51
119866,26.0,29.0,43.0,43.0,34.0,36.0,42.0,32.0,48.0,42.0,...,48.0,43.0,34.0,29.0,55.0,44.0,42.0,68.0,35.0,39
442876,122.0,120.0,131.0,86.0,92.0,82.0,105.0,101.0,104.0,114.0,...,111.0,94.0,103.0,120.0,81.0,81.0,91.0,105.0,98.0,108


In [31]:
X_valid

,sales(t-29),sales(t-28),sales(t-27),sales(t-26),sales(t-25),sales(t-24),sales(t-23),sales(t-22),sales(t-21),sales(t-20),...,sales(t-9),sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t)
117349,103.0,102.0,120.0,129.0,122.0,126.0,148.0,112.0,93.0,101.0,...,159.0,112.0,98.0,102.0,117.0,144.0,139.0,154.0,103.0,115
4370,97.0,95.0,87.0,109.0,68.0,74.0,79.0,94.0,83.0,65.0,...,60.0,77.0,72.0,68.0,56.0,41.0,52.0,45.0,60.0,73
45664,47.0,37.0,60.0,55.0,53.0,49.0,43.0,51.0,41.0,53.0,...,49.0,56.0,53.0,55.0,57.0,66.0,43.0,45.0,44.0,43
24414,42.0,41.0,38.0,42.0,51.0,54.0,51.0,41.0,41.0,49.0,...,52.0,36.0,43.0,36.0,47.0,51.0,58.0,55.0,36.0,39
36702,86.0,94.0,88.0,47.0,71.0,79.0,75.0,93.0,101.0,124.0,...,92.0,90.0,104.0,102.0,87.0,82.0,83.0,102.0,104.0,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99419,137.0,126.0,163.0,82.0,110.0,115.0,115.0,135.0,111.0,136.0,...,117.0,130.0,152.0,155.0,100.0,123.0,110.0,124.0,129.0,127
162285,105.0,75.0,73.0,87.0,92.0,80.0,89.0,100.0,69.0,62.0,...,75.0,82.0,65.0,77.0,84.0,73.0,78.0,96.0,89.0,71
22561,78.0,86.0,76.0,79.0,91.0,89.0,116.0,58.0,69.0,81.0,...,103.0,74.0,78.0,82.0,62.0,87.0,101.0,95.0,76.0,80
39174,99.0,106.0,82.0,97.0,114.0,89.0,104.0,104.0,126.0,76.0,...,98.0,113.0,121.0,80.0,92.0,88.0,92.0,100.0,127.0,107


In [45]:
mlp_train_pred = model_mlp.predict(X_train.values)
mlp_valid_pred = model_mlp.predict(X_valid.values)
print('Train rmse:', np.sqrt(mean_squared_error(Y_train, mlp_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_valid, mlp_valid_pred)))

Train rmse: 12.78625616693127
Validation rmse: 12.850449514995047


In [46]:
def mean_absolute_percentage_error(y_true, y_pred): 

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [47]:
Y_valid

[108.0,
 57.0,
 73.0,
 45.0,
 64.0,
 59.0,
 89.0,
 80.0,
 79.0,
 21.0,
 63.0,
 72.0,
 21.0,
 35.0,
 51.0,
 77.0,
 73.0,
 66.0,
 51.0,
 33.0,
 88.0,
 48.0,
 54.0,
 121.0,
 78.0,
 95.0,
 34.0,
 34.0,
 57.0,
 88.0,
 9.0,
 30.0,
 30.0,
 98.0,
 58.0,
 41.0,
 36.0,
 41.0,
 66.0,
 50.0,
 23.0,
 52.0,
 49.0,
 104.0,
 82.0,
 76.0,
 18.0,
 35.0,
 93.0,
 32.0,
 31.0,
 16.0,
 119.0,
 21.0,
 87.0,
 90.0,
 46.0,
 18.0,
 28.0,
 23.0,
 127.0,
 62.0,
 40.0,
 47.0,
 66.0,
 72.0,
 36.0,
 73.0,
 70.0,
 147.0,
 82.0,
 67.0,
 63.0,
 32.0,
 41.0,
 156.0,
 20.0,
 26.0,
 97.0,
 60.0,
 100.0,
 86.0,
 60.0,
 31.0,
 64.0,
 33.0,
 46.0,
 62.0,
 73.0,
 57.0,
 74.0,
 106.0,
 20.0,
 34.0,
 60.0,
 38.0,
 83.0,
 120.0,
 61.0,
 69.0,
 32.0,
 25.0,
 35.0,
 77.0,
 8.0,
 10.0,
 101.0,
 104.0,
 37.0,
 32.0,
 49.0,
 72.0,
 121.0,
 55.0,
 89.0,
 15.0,
 20.0,
 90.0,
 57.0,
 22.0,
 166.0,
 41.0,
 25.0,
 26.0,
 34.0,
 52.0,
 16.0,
 33.0,
 26.0,
 63.0,
 43.0,
 35.0,
 81.0,
 60.0,
 22.0,
 61.0,
 112.0,
 22.0,
 62.0,
 147.0,
 52.0,

In [38]:
Y_valid = Y_valid.ravel().tolist()

In [59]:
Y = np.array(Y_valid)
type(Y)

numpy.ndarray

In [54]:
len(Y_valid)


67188

In [57]:
mlp_valid_pred

[119.45610046386719,
 55.82282257080078,
 53.404781341552734,
 47.25389862060547,
 74.30168151855469,
 57.7198486328125,
 84.65180969238281,
 97.83497619628906,
 68.60587310791016,
 40.38874816894531,
 55.430606842041016,
 79.7592544555664,
 22.797607421875,
 40.54428482055664,
 42.83601379394531,
 105.13359069824219,
 81.4989013671875,
 75.7005844116211,
 49.4586296081543,
 45.546268463134766,
 113.37025451660156,
 43.14451217651367,
 48.25204849243164,
 141.36041259765625,
 74.62887573242188,
 108.28801727294922,
 29.33505630493164,
 48.88279342651367,
 93.15298461914062,
 64.95594024658203,
 20.086458206176758,
 38.28713607788086,
 25.095645904541016,
 79.93032836914062,
 50.077903747558594,
 39.679466247558594,
 42.07896041870117,
 37.40024948120117,
 79.33818817138672,
 48.078651428222656,
 35.30425262451172,
 64.12943267822266,
 52.63288497924805,
 70.95532989501953,
 71.18741607666016,
 84.70097351074219,
 20.945215225219727,
 39.1457633972168,
 77.75946807861328,
 44.9102592468

In [58]:
mlp = np.array(mlp_valid_pred)
type(mlp)

numpy.ndarray

In [55]:
len(mlp_valid_pred)

67188

In [50]:
mlp_valid_pred = mlp_valid_pred.ravel().tolist()

In [61]:
mean_absolute_percentage_error(Y,mlp)

20.039035034023797

In [126]:
from sklearn.metrics import mean_absolute_error
print('Train mape:', mean_absolute_error(Y_train,mlp_train_pred))

Train mape: 9.66199953660009


In [151]:
train_18 = train[train['date'] >= '2017-11-02']

In [152]:
train_18

,date,store,item,sales
1766,2017-11-02,1,1,21
1767,2017-11-03,1,1,18
1768,2017-11-04,1,1,34
1769,2017-11-05,1,1,23
1770,2017-11-06,1,1,17
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [153]:
train_18.head(35)

,date,store,item,sales
1766,2017-11-02,1,1,21
1767,2017-11-03,1,1,18
1768,2017-11-04,1,1,34
1769,2017-11-05,1,1,23
1770,2017-11-06,1,1,17
1771,2017-11-07,1,1,23
1772,2017-11-08,1,1,14
1773,2017-11-09,1,1,21
1774,2017-11-10,1,1,19
1775,2017-11-11,1,1,24


In [154]:
train_18_gp = train_18.sort_values('date').groupby(['item', 'store', 'date'], as_index=False)
train_18_gp = train_18_gp.agg({'sales':['mean']})
train_18_gp.columns = ['item', 'store', 'date', 'sales']
train_18_gp.head()

,item,store,date,sales
0,1,1,2017-11-02,21
1,1,1,2017-11-03,18
2,1,1,2017-11-04,34
3,1,1,2017-11-05,23
4,1,1,2017-11-06,17


In [155]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [156]:
window = 29
lag = lag_size
series = series_to_supervised(train_18.drop('date', axis=1), window=window, lag=lag)
series.head()

,store(t-29),item(t-29),sales(t-29),store(t-28),item(t-28),sales(t-28),store(t-27),item(t-27),sales(t-27),store(t-26),...,sales(t-2),store(t-1),item(t-1),sales(t-1),store(t),item(t),sales(t),store(t+30),item(t+30),sales(t+30)
1795,1.0,1.0,21.0,1.0,1.0,18.0,1.0,1.0,34.0,1.0,...,28.0,1.0,1.0,15.0,1,1,19,1.0,1.0,23.0
1796,1.0,1.0,18.0,1.0,1.0,34.0,1.0,1.0,23.0,1.0,...,15.0,1.0,1.0,19.0,1,1,16,2.0,1.0,35.0
1797,1.0,1.0,34.0,1.0,1.0,23.0,1.0,1.0,17.0,1.0,...,19.0,1.0,1.0,16.0,1,1,31,2.0,1.0,33.0
1798,1.0,1.0,23.0,1.0,1.0,17.0,1.0,1.0,23.0,1.0,...,16.0,1.0,1.0,31.0,1,1,7,2.0,1.0,46.0
1799,1.0,1.0,17.0,1.0,1.0,23.0,1.0,1.0,14.0,1.0,...,31.0,1.0,1.0,7.0,1,1,20,2.0,1.0,38.0


In [63]:
last_item = 'item(t-%d)' % window
last_store = 'store(t-%d)' % window
series = series[(series['store(t)'] == series[last_store])]
series = series[(series['item(t)'] == series[last_item])]

In [64]:
series

,item(t-29),store(t-29),sales(t-29),item(t-28),store(t-28),sales(t-28),item(t-27),store(t-27),sales(t-27),item(t-26),...,sales(t-2),item(t-1),store(t-1),sales(t-1),item(t),store(t),sales(t),item(t+90),store(t+90),sales(t+90)
29,1.0,1.0,19.0,1.0,1.0,15.0,1.0,1.0,10.0,1.0,...,16.0,1.0,1.0,24.0,1,1,9,1.0,1.0,33.0
30,1.0,1.0,15.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,...,24.0,1.0,1.0,9.0,1,1,17,1.0,1.0,15.0
31,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,14.0,1.0,...,9.0,1.0,1.0,17.0,1,1,15,1.0,1.0,21.0
32,1.0,1.0,16.0,1.0,1.0,14.0,1.0,1.0,24.0,1.0,...,17.0,1.0,1.0,15.0,1,1,17,1.0,1.0,29.0
33,1.0,1.0,14.0,1.0,1.0,24.0,1.0,1.0,14.0,1.0,...,15.0,1.0,1.0,17.0,1,1,24,1.0,1.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182405,50.0,10.0,79.0,50.0,10.0,89.0,50.0,10.0,96.0,50.0,...,79.0,50.0,10.0,80.0,50,10,82,50.0,10.0,63.0
182406,50.0,10.0,89.0,50.0,10.0,96.0,50.0,10.0,92.0,50.0,...,80.0,50.0,10.0,82.0,50,10,90,50.0,10.0,59.0
182407,50.0,10.0,96.0,50.0,10.0,92.0,50.0,10.0,102.0,50.0,...,82.0,50.0,10.0,90.0,50,10,103,50.0,10.0,74.0
182408,50.0,10.0,92.0,50.0,10.0,102.0,50.0,10.0,60.0,50.0,...,90.0,50.0,10.0,103.0,50,10,99,50.0,10.0,62.0


,item,store,date,sales
0,1,1,2017-01-01,19
1,1,1,2017-01-02,15
2,1,1,2017-01-03,10
3,1,1,2017-01-04,16
4,1,1,2017-01-05,14
